In [22]:
import numpy as np
import pandas as pd

In [23]:
df = pd.read_csv('Bank_Personal_Loan_Modelling.csv')
df = df.drop(['ID', 'ZIP Code'], axis=1)
df = pd.get_dummies(df, columns=['Education', 'Personal Loan', 'Securities Account', 'CD Account', 'Online', 'CreditCard'],drop_first=True)
x = df.drop('Personal Loan_1', axis=1)
y = df['Personal Loan_1'].values

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [25]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [27]:
input_dim = x_train.shape[1]
hidden_dim = 5
output_dim = 1

In [28]:
w1 = np.random.randn(input_dim, hidden_dim)
w2 = np.random.randn(hidden_dim, output_dim)

In [29]:
def fitness_function(w):
    global w1, w2
    w1 = w[:input_dim * hidden_dim].reshape(input_dim, hidden_dim)
    w2 = w[input_dim * hidden_dim:].reshape(hidden_dim, output_dim)
    z1 = np.dot(x_train, w1)
    a1 = sigmoid(z1)
    z2 = np.dot(a1, w2)
    y_pred = sigmoid(z2)
    y_pred = np.round(y_pred)
    accuracy = np.mean(y_pred == y_train)
    return accuracy

In [36]:
def pso(fitness_function, num_particles, max_iter):
    positions = np.random.randn(num_particles, input_dim * hidden_dim + hidden_dim * output_dim).astype(float)
    velocities = np.zeros_like(positions)
    global_best_position = positions[0]
    global_best_fitness = 0
    for i in range(max_iter):
        for j in range(num_particles):
            inertia = 0.5
            cognitive = 1
            social = 1
            r1 = np.random.rand(input_dim * hidden_dim + hidden_dim * output_dim)
            r2 = np.random.rand(input_dim * hidden_dim + hidden_dim * output_dim)
            velocities[j] = inertia * velocities[j] + cognitive * r1 * (global_best_position - positions[j]) + social * r2 * (np.random.randn(input_dim * hidden_dim + hidden_dim * output_dim) - positions[j])
            positions[j] = positions[j] + velocities[j]
        fitness = np.array([fitness_function(positions[j]) for j in range(num_particles)])
        if fitness.max() > global_best_fitness:
            global_best_fitness = fitness.max()
            global_best_position = positions[fitness.argmax()]
    global_best_w1 = global_best_position[:input_dim * hidden_dim].reshape(input_dim, hidden_dim)
    global_best_w2 = global_best_position[input_dim * hidden_dim:].reshape(hidden_dim, output_dim)
    return global_best_w1, global_best_w2

In [37]:
num_particles = 10
max_iter = 100
w1, w2 = pso(fitness_function, num_particles, max_iter)

In [39]:
z1 = np.dot(x_test, w1)
a1 = sigmoid(z1)
z2 = np.dot(a1, w2)
y_pred = sigmoid(z2)

In [40]:
y_pred = np.round(y_pred)
accuracy = np.mean(y_pred == y_test)
accuracy

0.8928